<a href="https://colab.research.google.com/github/thesalmonification/DSCI400_Revamp/blob/master/PhaseVocoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
############################################################
"""This code was designed to implement phase vocoding, a technique used
to shorten signals in time without destroying their frequency content. This
function is vital to our project as unaltered EEG signals varied in length for 
each session. Phase vocoding was implemented using the popular audio editing 
library librosa"""


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
%matplotlib inline

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Import the waveform of the HDF5
keys = pd.read_csv('/content/drive/Shared drives/DSCI400/Waveform_Data_Keys.csv',header=None)
keys = list(keys[0])

In [0]:
#iterate over all keys to grab all waveforms...

#I need to shorten all longer waveforms to be equal to the shortest of all sessions
#Therefore, I need to know the length of the shortest waveform. Let's find it...
shortest_length = float('inf')

for key in keys:
  print(key)
  session_df = pd.read_hdf('/content/drive/Shared drives/DSCI400/Waveform_Data.h5',key=key)
  session_length = session_df.shape[0]
  if session_length < shortest_length:
    shortest_length = session_length
print("Shortest Length is: " + str(shortest_length))


t10
t1042
t1044
t1046
t1048
t1050
t1052
t1054
t1056
t1058
t1060
t1062
t1064
t1066
t1068
t1172
t1174
t1176
t1178
t1182
t1184
t1186
t1188
t1190
t1192
t1194
t1196
t1198
t12
t1200
t1202
t1204
t1206
t1208
t1210
t1302
t1304
t1306
t2376
t2378
t2380
t24
t2472
t2474
t2476
t2478
t2480
t2482
t2484
t2486
t2488
t2490
t2492
t2494
t2496
t2498
t2500
t150
t152
t154
t156
t1562
t1564
t1566
t1568
t1570
t1572
t1574
t1576
t1578
t158
t1580
t1582
t1584
t1586
t1588
t1180
t1308
t148
t1590
t1730
t2082
t22
t2374
t2502
t2628
t2746
t2874
t2996
t3124
t3382
t3420
t3550
t3678
t402
t550
t680
t810
t1592
t1594
t1596
t1598
t16
t160
t1600
t162
t164
t166
t168
t1692
t1694
t1696
t1698
t170
t1700
t1702
t1704
t1706
t1708
t1710
t1712
t1714
t1716
t1718
t1720
t1722
t1724
t1726
t1728
t2504
t2506
t2508
t2510
t26
t2602
t2604
t2606
t2608
t2610
t2612
t2614
t2616
t2618
t262
t2620
t2622
t2624
t2626
t2084
t2086
t2088
t2090
t2092
t2094
t2096
t2098
t2100
t2102
t2104
t2106
t2108
t2110
t2112
t2114
t2116
t2118
t2120
t3512
t3514
t3516
t3518
t35

In [0]:
#Now for every key, phase vocode every channel and save to a new HDF5 file...

for key in keys:
  print(key)
  session_df = pd.read_hdf('/content/drive/Shared drives/DSCI400/Waveform_Data.h5',key=key)
  session_length = session_df.shape[0]
  
  #Here I need to calculate the phase vocoding fraction to shorten any longer waveforms...
  fraction = session_length / shortest_length


  #Build the first vocoded dataframe so I may append to it
  waveform = session_df[session_df.columns[0]].to_numpy()
  STFT = librosa.stft(waveform,256)
  STFT_fast = librosa.phase_vocoder(STFT, fraction)
  waveform_fast = librosa.istft(STFT_fast)

  waveform_array = waveform_fast
  waveform_array = waveform_array.reshape(-1,1).T


  #Iterate over all others chs to build the shortened waveform array
  for ch in session_df.columns[1:]:
    waveform = session_df[ch].to_numpy()


    #Here I do vocoding thru a STFT and the librosa phase vocoder function...
    STFT = librosa.stft(waveform,256)
    STFT_fast = librosa.phase_vocoder(STFT, fraction)
    waveform_fast = librosa.istft(STFT_fast)
    waveform_fast = waveform_fast.reshape(-1,1).T

    #Append to the growing waveform array
    waveform_array = np.concatenate((waveform_array,waveform_fast),axis=0)

  #Save waveform to New HDF5 file...
  waveform_df = pd.DataFrame(waveform_array,index=session_df.columns).T

  waveform_df.to_hdf('/content/drive/Shared drives/DSCI400/Waveform_Vocoded_Unequal_Data.h5',key=key,complevel=4,format='table')


     

t10
t1042
t1044
t1046
t1048
t1050
t1052
t1054
t1056
t1058
t1060
t1062
t1064
t1066
t1068
t1172
t1174
t1176
t1178
t1182
t1184
t1186
t1188
t1190
t1192
t1194
t1196
t1198
t12
t1200
t1202
t1204
t1206
t1208
t1210
t1302
t1304
t1306
t2376
t2378
t2380
t24
t2472
t2474
t2476
t2478
t2480
t2482
t2484
t2486
t2488
t2490
t2492
t2494
t2496
t2498
t2500
t150
t152
t154
t156
t1562
t1564
t1566
t1568
t1570
t1572
t1574
t1576
t1578
t158
t1580
t1582
t1584
t1586
t1588
t1180
t1308
t148
t1590
t1730
t2082
t22
t2374
t2502
t2628
t2746
t2874
t2996
t3124
t3382
t3420
t3550
t3678
t402
t550
t680
t810
t1592
t1594
t1596
t1598
t16
t160
t1600
t162
t164
t166
t168
t1692
t1694
t1696
t1698
t170
t1700
t1702
t1704
t1706
t1708
t1710
t1712
t1714
t1716
t1718
t1720
t1722
t1724
t1726
t1728
t2504
t2506
t2508
t2510
t26
t2602
t2604
t2606
t2608
t2610
t2612
t2614
t2616
t2618
t262
t2620
t2622
t2624
t2626
t2084
t2086
t2088
t2090
t2092
t2094
t2096
t2098
t2100
t2102
t2104
t2106
t2108
t2110
t2112
t2114
t2116
t2118
t2120
t3512
t3514
t3516
t3518
t35

In [0]:
#Since the vocoder isn't perfect, reopen the file and shorten all to the newest resulting shortest value...


shortest_length = float('inf')

for key in keys:
  print(key)
  session_df = pd.read_hdf('/content/drive/Shared drives/DSCI400/Waveform_Vocoded_Unequal_Data.h5',key=key)
  session_length = session_df.shape[0]
  if session_length < shortest_length:
    shortest_length = session_length

print("Shortest Length is: " + str(shortest_length))

t10
t1042
t1044
t1046
t1048
t1050
t1052
t1054
t1056
t1058
t1060
t1062
t1064
t1066
t1068
t1172
t1174
t1176
t1178
t1182
t1184
t1186
t1188
t1190
t1192
t1194
t1196
t1198
t12
t1200
t1202
t1204
t1206
t1208
t1210
t1302
t1304
t1306
t2376
t2378
t2380
t24
t2472
t2474
t2476
t2478
t2480
t2482
t2484
t2486
t2488
t2490
t2492
t2494
t2496
t2498
t2500
t150
t152
t154
t156
t1562
t1564
t1566
t1568
t1570
t1572
t1574
t1576
t1578
t158
t1580
t1582
t1584
t1586
t1588
t1180
t1308
t148
t1590
t1730
t2082
t22
t2374
t2502
t2628
t2746
t2874
t2996
t3124
t3382
t3420
t3550
t3678
t402
t550
t680
t810
t1592
t1594
t1596
t1598
t16
t160
t1600
t162
t164
t166
t168
t1692
t1694
t1696
t1698
t170
t1700
t1702
t1704
t1706
t1708
t1710
t1712
t1714
t1716
t1718
t1720
t1722
t1724
t1726
t1728
t2504
t2506
t2508
t2510
t26
t2602
t2604
t2606
t2608
t2610
t2612
t2614
t2616
t2618
t262
t2620
t2622
t2624
t2626
t2084
t2086
t2088
t2090
t2092
t2094
t2096
t2098
t2100
t2102
t2104
t2106
t2108
t2110
t2112
t2114
t2116
t2118
t2120
t3512
t3514
t3516
t3518
t35

In [0]:
#Save to a final file that uses the shortest length of all files...

for key in keys:
  session_df = pd.read_hdf('/content/drive/Shared drives/DSCI400/Waveform_Vocoded_Unequal_Data.h5',key=key)
  
  session_df.to_hdf('/content/drive/Shared drives/DSCI400/Waveform_Vocoded_Equalized_Data.h5',key=key,complevel=4,format='table')
